In [1]:
!python --version

Python 3.10.12


In [ ]:
# !pip install git+https://github.com/dClimate/ipldstore.git@ipfs-benchmarking
!pip install git+https://github.com/dmp267/ipldstore.git@ipfs-benchmarking

  Cloning https://github.com/dmp267/ipldstore.git (to revision ipfs-benchmarking) to /tmp/pip-req-build-c84h_kfu
  Running command git clone --filter=blob:none --quiet https://github.com/dmp267/ipldstore.git /tmp/pip-req-build-c84h_kfu
  Running command git checkout -b ipfs-benchmarking --track origin/ipfs-benchmarking
  Switched to a new branch 'ipfs-benchmarking'
  Branch 'ipfs-benchmarking' set up to track remote branch 'ipfs-benchmarking' from 'origin'.
  Resolved https://github.com/dmp267/ipldstore.git to commit 153d7c123e3bbc22bd7d6a2d2569e8c6c14b0c87
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/dClimate/py-hamt (to revision v1.1.0) to /tmp/pip-install-nara8tyq/py-hamt_870c571e87c14bcf8490e9c4d8c376bb
  Running command git clone --filter=blob:none --quiet https://github.com/dClimate/py-hamt /tmp/pip-install-nara8tyq/py-hamt_870c571e87c14bcf8490e9c4d8c376bb
  Runn

In [ ]:
# install ipldstore_v1
!pip install git+https://github.com/dmp267/ipldstore-v1.git

In [ ]:
!pip install xarray

In [ ]:
!pip install nest_asyncio

In [ ]:
!pip install matplotlib

In [ ]:
# updated to most recent version 0.20 -> 0.26
!wget https://dist.ipfs.tech/kubo/v0.26.0/kubo_v0.26.0_linux-amd64.tar.gz
!tar -xvzf kubo_v0.26.0_linux-amd64.tar.gz

In [ ]:
!bash /content/kubo/install.sh

In [ ]:
!ipfs --version

In [ ]:
!ipfs init

In [ ]:
# if this cell doesn't complete within a couple seconds just restart it
# if it still doesn't complete within a couple seconds, try the next cell
# once or twice then try this one again
import os
import time

!ipfs config Addresses.Gateway /ip4/127.0.0.1/tcp/8082
get_ipython().system_raw('ipfs daemon &')

ready = os.popen('ipfs stats bw').read()
print(ready)
while not ready:
  time.sleep(0.1)
  ready = os.popen('ipfs stats bw').read()

print('ipfs is ready')

In [ ]:
# empty garbage
!ipfs repo gc

In [ ]:
import json

print(f'Manually setting HAMT CID')
hamt_metadata_cid = "bafyreicvczjixk5g7gs4rdd3sjvt7wab7mqoqcj6mqkh3fq7rnpoyc5ati"
print(hamt_metadata_cid)

In [ ]:
print(f'Connecting to Zarr node')
!ipfs swarm connect /ip4/45.55.32.80/tcp/4001/p2p/12D3KooWG7itEPAHut3xsVo7CwyD8sKeQXKgQizotNhPsToCssXQ

In [ ]:
# get era5_precip CID for non-HAMT releases
metadata_cid = !ipfs name resolve k2k4r8l6plbm4r3ks757u8avo5n8t0ghszkwx3uu61cid363lx45n5kl
metadata_cid = metadata_cid[0].split('/')[2]
!ipfs dag get {metadata_cid} | python3 -m json.tool

metadata = !ipfs dag get {metadata_cid}

zarr_metadata_cid = json.loads(metadata[0])["assets"]["zmetadata"]["href"]["/"]
print(zarr_metadata_cid)

In [ ]:
print(f'Connecting to HAMT node')
!ipfs swarm connect /ip4/167.71.180.28/tcp/4001/p2p/12D3KooWFqzYkjofVzvo7MdYy4h3cZwixJC1f6NFWGYPELU7yBNy

In [ ]:
# NOTICE: STOP executing here and skip ahead 8 cells
# to test non-HAMT retrieval only
import ipldstore
import xarray as xr
import nest_asyncio
nest_asyncio.apply()

async def hamt():
    m = ipldstore.get_ipfs_mapper(
      host = "http://127.0.0.1:5001",
      max_nodes_per_level = 10000,
      chunker = "size-262144",
      should_async_get = True,
    )

    m.set_root(hamt_metadata_cid)

    ds = xr.open_zarr(m, chunks=None)
    return ds

print('Testing HAMT')
xar = await hamt()

In [ ]:
xar

In [ ]:
%time x = xar.sel(latitude=40.25, longitude=-120.25, time=slice("2005-01-01", "2010-12-31")).tp.values

In [ ]:
# test cached speed
%time x = xar.sel(latitude=40.25, longitude=-120.25, time=slice("2005-01-01", "2010-12-31")).tp.values

In [ ]:
# confirm data size
number_bytes = xar.sel(latitude=40.25, longitude=-120.25, time=slice("2005-01-01", "2010-12-31")).nbytes
number_bytes

In [ ]:
# confirm data visually
import matplotlib.pyplot as plt

query = xar.sel(latitude=40.25, longitude=-120.25, time=slice("2005-01-01", "2010-12-31"))
plt.rcParams['figure.figsize'] = [14, 7]
plt.plot(query.time.values, query.tp.values)

In [ ]:
# empty garbage
!ipfs repo gc

In [ ]:
# refresh connection to Arbol pre-HAMT Zarr peer
!ipfs swarm connect /ip4/45.55.32.80/tcp/4001/p2p/12D3KooWG7itEPAHut3xsVo7CwyD8sKeQXKgQizotNhPsToCssXQ

In [ ]:
import ipldstore_v1
import xarray as xr
import nest_asyncio
nest_asyncio.apply()

async def zarr():
    m = ipfs_mapper = ipldstore_v1.get_ipfs_mapper(
      host = "http://127.0.0.1:5001",
      max_nodes_per_level = 10000,
      chunker = "size-262144",
      should_async_get = True,
    )

    m.set_root(zarr_metadata_cid)

    ds = xr.open_zarr(m, chunks=None)
    return ds

print('Testing non-HAMT Zarr')
zar = await zarr()

In [ ]:
zar

In [ ]:
%time z = zar.sel(latitude=40.25, longitude=-120.25, time=slice("2005-01-01", "2010-12-31")).tp.values

In [ ]:
# retest cached speed
%time z = zar.sel(latitude=40.25, longitude=-120.25, time=slice("2005-01-01", "2010-12-31")).tp.values

In [ ]:
number_bytes = zar.sel(latitude=40.25, longitude=-120.25, time=slice("2005-01-01", "2010-12-31")).nbytes
number_bytes

In [ ]:
import matplotlib.pyplot as plt

query = zar.sel(latitude=40.25, longitude=-120.25, time=slice("2005-01-01", "2010-12-31"))
plt.rcParams['figure.figsize'] = [14, 7]
plt.plot(query.time.values, query.tp.values)